# Quick exploration of geovec, a pre-trained word embedding in gloVe format

#### Data file of the pretrained embedding is downloaded from these links:

download site & information https://osf.io/4uyeq/wiki/home/

h5 file = https://osf.io/4uyeq/

Medium article by authors: https://towardsdatascience.com/geovec-word-embeddings-for-geosciences-ac1e1e854e19

In [1]:
#conda install -c anaconda scikit-learn 
# !pip install h5py
#!conda install -c anaconda h5py 

In [2]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
#from sklearn.manifold import TSNE

In [3]:
import json
import h5py
import numpy as np

##### glove python is a dependency mentioend by the authors
##### DON'T DO THIS AS IT HAS AlL SORTS OF PROBLEMS ON MACOS AND WINDOWS

#!pip install glove_python

### Instead of using glove-python, we can just open the file this way

In [4]:
with h5py.File('geovec_300d_v1.h5', 'r') as f:
        v = np.zeros(f['vectors'].shape, f['vectors'].dtype)
        f['vectors'].read_direct(v)
        dct = f['dct'][()].tostring().decode('utf-8')
        dct = json.loads(dct)

In [5]:
#### number of words with vector embeddings
print("number of words with vector embeddings",len(v))
#### Length of vectors
print("number of vectors in each embedding",len(v[1]))

number of words with vector embeddings 411845
number of vectors in each embedding 300


In [6]:
print("number of words total",len(dct))
print("and what each word in dictionary looks like: ",list(dct.keys())[0:1][0],":",dct[list(dct.keys())[0:1][0]])

number of words total 411845
and what each word in dictionary looks like:  shinan : 224680


In [7]:
print("embedding for acrodermatitis",v[dct[list(dct.keys())[0:1][0]]])

embedding for acrodermatitis [-0.10594086 -0.05738851  0.03194365 -0.01788764  0.00262375  0.01449635
 -0.02291972  0.05872495  0.14296882 -0.02410946 -0.0664839   0.06400142
 -0.04727975  0.11105183 -0.00495148  0.02542379  0.04844313 -0.03132289
 -0.07447407 -0.04403799 -0.04219329  0.02730621  0.02795844  0.04235579
 -0.04048587 -0.11405727  0.02757259 -0.05587896 -0.0035318   0.02695823
 -0.04781603  0.08963217 -0.08274658  0.0417579  -0.01025344  0.09871103
  0.02191398  0.04148114 -0.0323418   0.07600801  0.0348746  -0.06986226
 -0.05228152  0.08611795 -0.03221705  0.10128982 -0.04551369 -0.02769331
  0.01034517 -0.0101576  -0.13007206 -0.01927234  0.11720682 -0.04032579
  0.1619189   0.12280071 -0.03354213  0.02586541  0.02279903  0.00185333
  0.04232504  0.06906866 -0.01626612 -0.02051888 -0.07304693  0.06525084
 -0.02131874  0.08204588  0.10155227 -0.02897052  0.09239195 -0.08750017
  0.09984715  0.03835623  0.06661657  0.04644443  0.11996252 -0.08092465
 -0.09771429  0.133809

In [8]:
list(dct.keys())[0:10]

['shinan',
 'isocyanic',
 'pnc',
 'sichuan',
 'curium',
 'iddings',
 'antidiarrheal',
 'hilomonadea',
 'gier',
 'gyda']

In [9]:
type(v)

numpy.ndarray

In [10]:
#v

### We'll combine the two files into a single dictionary to make some later things a bit easier. It isn't saved in thi format to save space according to the authors who trained it

In [11]:
def replaceIntsWithWords(vectors,dictionary):
    words = list(dct.keys())
    embeddings_dict = {}
    for word in words:
        embeddings_dict[word] = np.asarray(v[dictionary[word]][:],"float32")
    return embeddings_dict

In [12]:
new_vector_embedding = replaceIntsWithWords(v,dct)

In [13]:
new_vector_embedding["sandstone"]

array([ 0.03063519, -0.41190132, -0.01748249, -0.03562951, -0.44922313,
       -0.23300318,  0.22224723, -0.07885347,  0.02758265, -0.07660752,
       -0.39882764,  0.19357805, -0.03862172, -0.16392523, -0.08809105,
       -0.15323128, -0.17489772,  0.14711891,  0.10426124, -0.02329986,
        0.09977868,  0.23435533,  0.3037015 , -0.6652086 , -0.1779219 ,
        0.16997509, -0.0749488 , -0.4623773 , -0.18799299, -0.04015233,
       -0.1230348 , -0.324141  ,  0.32538635,  0.27751258, -0.04844321,
        0.04106978,  0.3250498 ,  0.4131149 ,  0.41197386,  0.05734786,
       -0.28493047,  0.05330373, -0.30320197, -0.6228066 ,  0.17232999,
        0.13726279,  0.02862607,  0.00987866, -0.22104132, -0.27486506,
        0.3875012 ,  0.05712619,  0.30534878, -0.17705044, -0.19142899,
       -0.1692242 , -0.3759296 ,  0.35963646, -0.02505949,  0.05952422,
        0.33059987,  0.18033917,  0.05865888,  0.1908917 ,  0.03191795,
       -0.3452011 , -0.4242747 ,  0.02242155, -0.08702113, -0.36

### Now let's play around a bit to find out what words are close to others in the embedding space

In [14]:
def find_closest_embeddings(embeddings_dict,embedding_vector):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding_vector))

In [15]:
len(new_vector_embedding.keys())

411845

In [16]:
spatial.distance.euclidean(new_vector_embedding['sandstone'], new_vector_embedding['granite'])

4.911217212677002

In [17]:
find_closest_embeddings(new_vector_embedding,new_vector_embedding["sandstone"])[0:10]

['sandstone',
 'siltstone',
 'mudstone',
 'arkosic',
 'conglomerate',
 'rothbach',
 'gröden',
 'sonta',
 'mudstones',
 'interbedded']

In [18]:
find_closest_embeddings(new_vector_embedding,new_vector_embedding["granite"])[0:10]

['granite',
 'granitic',
 'ebat',
 'tuaní',
 'kibe',
 'lavadores',
 'baerzhe',
 'daguzhai',
 'cooyerdoo',
 'qôrqut']

In [19]:
find_closest_embeddings(new_vector_embedding,new_vector_embedding["crossbedding"])[0:20]

['crossbedding',
 'lumi',
 'unrein',
 'superface',
 'boundinaged',
 'staretsky',
 'essikado',
 'gerlarch',
 'doucette',
 'szenleng',
 'rhune',
 'shihui',
 'gackowa',
 'lenticularity',
 'tenappera',
 'karlshafen',
 'håkjerringdjupet',
 'anorbital',
 'eludicate',
 'torom']

### Looks like this is a lot of non-English words?!!

### Let's try subtraction of pure vectors

In [20]:
print(find_closest_embeddings(new_vector_embedding,
    new_vector_embedding["stratum"] - new_vector_embedding["fault"]
)[:25])

['sagyeri', 'pregrowth', 'barzaloza', 'translatent', 'syntransform', 'dothan', 'superincumbent', 'corniola', 'adoudou', 'chingmei', 'březno', 'saberian', 'kulm', 'trentinara', 'flamy', 'chulca', 'setlaole', 'emborozu', 'ikpe', 'guxu', 'aib', 'sarsbukta', 'huilu', 'nunda', 'lecho']


In [21]:
print(find_closest_embeddings(new_vector_embedding,
    new_vector_embedding["marble"] - new_vector_embedding["heat"]
)[:25])

['marble', 'sedef', 'pentelic', 'supren', 'macael', 'yeosan', 'calanchiole', 'pentelikon', 'kemalpasa', 'dionysos', 'borba', 'alabashka', 'iscehisar', 'maniçoba', 'hekkelstrand', 'benou', 'calcsilicates', 'evenes', 'harsin', 'montepuez', 'ayhan', 'nanshu', 'sanchuan', 'belevi', 'bettadabidu']


In [22]:
print(find_closest_embeddings(new_vector_embedding,
    new_vector_embedding["boulder"] - new_vector_embedding["clay"]
)[:25])

['intratill', 'satitoa', 'boulder', 'montoz', 'laxe', 'kintradwell', 'unclimbed', 'settai', 'norber', 'alveolized', 'intertill', 'cipit', 'preexposure', 'kudaka', 'moutonnee', 'ventifacted', 'navabandar', 'sturgis', 'taufererberg', 'pineo', 'preexposed', 'mathu', 'runswick', 'spelonk', 'encierro']


### A blunt approach doesn't work so well

## Let's try to use tensorflow's embedding projector

In [23]:
!pip install torch

In [24]:
!pip install fasttext

https://stackoverflow.com/questions/41258391/tensorboard-embedding-example/41262360#41262360

In [25]:
import os
import tensorflow as tf
import numpy as np
#import fasttext
from tensorflow.contrib.tensorboard.plugins import projector



In [26]:
tf

<module 'tensorflow' from '/Users/justingosses/anaconda/lib/python3.5/site-packages/tensorflow/__init__.py'>

In [27]:

# # load model
# word2vec = fasttext.load_model('wiki.en.bin')
# # load model
# #word2vec = fasttext.load_model('wiki.en.bin')

In [28]:
len(v[0])

300

In [29]:
len(dct)

411845

In [30]:
list(dct.keys())

['shinan',
 'isocyanic',
 'pnc',
 'sichuan',
 'curium',
 'iddings',
 'antidiarrheal',
 'hilomonadea',
 'gier',
 'gyda',
 'naranjos',
 'guangjin',
 'chommal',
 'pseudogene',
 'haningtoni',
 'ospo',
 'chainpur',
 'enacts',
 'unparalled',
 'termohaline',
 'orthopteroids',
 'dave',
 'ferrigehlenite',
 'certified',
 'laminata',
 'skeletal',
 'arikawa',
 'beachgrass',
 'depicting',
 'kuechulin',
 'lowenstami',
 'nguène',
 'schistic',
 'arsenicoselenatis',
 'biologica',
 'melbtal',
 'chake',
 'manceau',
 'condolios',
 'lappel',
 'nonmineralized',
 'smacs',
 'lagrangians',
 'symmetrodonts',
 'tizgui',
 'inclusiveness',
 'perspectivalism',
 'decorticate',
 'mathews',
 'hantam',
 'conidiogenous',
 'ranipes',
 'lièvres',
 'labouchere',
 'wulan',
 'immunoperoxidase',
 'laplace',
 'mayne',
 'zzz',
 'seismostruct',
 'impairment',
 'tubercules',
 'pectens',
 'neublans',
 'syringae',
 'kwanak',
 'mfth',
 'canoochee',
 'methanoregula',
 'eighe',
 'contaminée',
 'précurseurs',
 'prieto',
 'protégée',
 '

In [31]:
v

array([[ 0.6575543 , -0.10153764, -0.2825202 , ...,  0.64620304,
         0.3083481 , -0.46784   ],
       [ 0.06665467,  0.11548886,  0.22080564, ..., -0.08355215,
         0.00378662, -0.13758856],
       [ 0.32676426,  0.26151726, -0.33332917, ...,  0.15112405,
         0.00892717, -0.19385871],
       ...,
       [ 0.02525451, -0.00096976,  0.04692964, ..., -0.05422716,
        -0.02048381,  0.01153759],
       [ 0.09515277,  0.09372371,  0.09677262, ..., -0.10224963,
        -0.06350729, -0.05045411],
       [-0.02760863,  0.0040131 ,  0.03059224, ..., -0.04461955,
        -0.03546656, -0.00330813]], dtype=float32)

In [32]:
# # create a list of vectors
embedding = np.empty((len(dct), len(v[0])), dtype=np.float32)
i = 0
words = list(dct.keys())
for word in words:
    embedding[i] = v[dct[word]]
    i+=1



In [33]:
embedding

array([[-0.10594086, -0.05738851,  0.03194365, ..., -0.0674778 ,
        -0.0818163 ,  0.03489417],
       [ 0.04156372,  0.04673143,  0.00152126, ..., -0.04678056,
        -0.04335489,  0.06677914],
       [ 0.15134504, -0.15312792,  0.22095971, ...,  0.1411554 ,
         0.05863498, -0.08714896],
       ...,
       [ 0.01521935,  0.01039202,  0.07480428, ..., -0.11425154,
        -0.01520954,  0.02502503],
       [-0.03544094, -0.01141176,  0.04448843, ..., -0.09215635,
        -0.04700284, -0.00754375],
       [-0.02017696, -0.04253832,  0.03035418, ..., -0.07796827,
        -0.01703196,  0.00919739]], dtype=float32)

In [34]:
words

['shinan',
 'isocyanic',
 'pnc',
 'sichuan',
 'curium',
 'iddings',
 'antidiarrheal',
 'hilomonadea',
 'gier',
 'gyda',
 'naranjos',
 'guangjin',
 'chommal',
 'pseudogene',
 'haningtoni',
 'ospo',
 'chainpur',
 'enacts',
 'unparalled',
 'termohaline',
 'orthopteroids',
 'dave',
 'ferrigehlenite',
 'certified',
 'laminata',
 'skeletal',
 'arikawa',
 'beachgrass',
 'depicting',
 'kuechulin',
 'lowenstami',
 'nguène',
 'schistic',
 'arsenicoselenatis',
 'biologica',
 'melbtal',
 'chake',
 'manceau',
 'condolios',
 'lappel',
 'nonmineralized',
 'smacs',
 'lagrangians',
 'symmetrodonts',
 'tizgui',
 'inclusiveness',
 'perspectivalism',
 'decorticate',
 'mathews',
 'hantam',
 'conidiogenous',
 'ranipes',
 'lièvres',
 'labouchere',
 'wulan',
 'immunoperoxidase',
 'laplace',
 'mayne',
 'zzz',
 'seismostruct',
 'impairment',
 'tubercules',
 'pectens',
 'neublans',
 'syringae',
 'kwanak',
 'mfth',
 'canoochee',
 'methanoregula',
 'eighe',
 'contaminée',
 'précurseurs',
 'prieto',
 'protégée',
 '

In [35]:
# # create a list of vectors
# embedding = np.empty((len(word2vec.words), word2vec.dim), dtype=np.float32)
# for i, word in enumerate(word2vec.words):
#     embedding[i] = word2vec[word]

In [36]:
!mkdir 'log'
!cd log
!touch 'metadata.tsv'
!cd ..

mkdir: log: File exists


In [138]:
# setup a TensorFlow session
tf.reset_default_graph()
sess = tf.InteractiveSession()
X = tf.Variable([0.0], name='embedding')
place = tf.placeholder(tf.float32, shape=embedding.shape)
set_x = tf.assign(X, place, validate_shape=False)
sess.run(tf.global_variables_initializer())
sess.run(set_x, feed_dict={place: embedding})

# write labels
# with open('log/metadata.tsv', 'w') as f:
# #     for word in word2vec.words:
#     for word in words:
#         f.write(word + '\n')

# create a TensorFlow summary writer
summary_writer = tf.summary.FileWriter('log', sess.graph)
config = projector.ProjectorConfig()
embedding_conf = config.embeddings.add()
embedding_conf.tensor_name = 'embedding:0'
embedding_conf.metadata_path = os.path.join('log', 'metadata1.tsv')
projector.visualize_embeddings(summary_writer, config)

# save the model
saver = tf.train.Saver()
saver.save(sess, os.path.join('log', "model.ckpt"))

'log/model.ckpt'

In [139]:
!tensorboard --logdir=log

TensorFlow installation not found - running with reduced feature set.
W0729 20:29:11.878477 123145570467840 plugin_event_accumulator.py:323] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0729 20:29:11.879190 123145570467840 plugin_event_accumulator.py:335] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0729 20:29:11.880587 123145570467840 plugin_event_accumulator.py:323] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0729 20:29:11.882354 123145570467840 plugin_event_accumulator.py:323] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0729 20:29:11.883895 12314557046

In [42]:
#!pip install mkl

In [40]:
import io


In [41]:

encoder = info.features['text'].encoder

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(encoder.subwords):
  vec = weights[num+1] # skip 0, it's padding.
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()


NameError: name 'info' is not defined

In [ ]:
!cd log
!touch 'log/vecs.tsv'
!cd ..

In [ ]:

out_v = io.open('log/vecs.tsv', 'w', encoding='utf-8')

for n,weights in enumerate(embedding):
    vec = v[n]
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")

### Get geology dictionary of english words
here: https://www.geologynet.com/programs/geology.dic

wget gets denied! just download manually

In [ ]:
# !pip install wget

In [ ]:
# import wget


In [ ]:
# geowords_url ='https://www.geologynet.com/programs/geology.dic'

In [ ]:
# geowords_english_r = requests.get(geowords_url)
# wget.download(geowords_ur, 'geowords.txt')


In [ ]:
# geowords_english_r

In [ ]:
# geowords_english = open('./geowords.txt', 'wb').write(geowords_english_r.content)

In [ ]:
# geowords_english

In [43]:
file = open('geowords.txt', 'r') 
geowords_english = file.read()

In [44]:
len(geowords_english)

56906

In [45]:
geowords_english[0:50]

'aalenian\nabaete\nabdon\naberdour\nabiogenic\nabiotic\na'

In [46]:
geowords_split = geowords_english.split("\n")

In [47]:
geowords_split[0:50]

['aalenian',
 'abaete',
 'abdon',
 'aberdour',
 'abiogenic',
 'abiotic',
 'abiquiu',
 'abitibi',
 'abk',
 'ablagerung',
 'ablagerungen',
 'ablagerungsmilieu',
 'ablagerungsmilieus',
 'aboral',
 'abrolhos',
 'abrupts',
 'abs',
 'absaroka',
 'absarokite',
 'abu',
 'abundances',
 'acadian',
 'acado',
 'acanthocephela',
 'accomack',
 'accor',
 'accretionary',
 'acheulian',
 'achladi',
 'achondrites',
 'achroite',
 'acicular',
 'acoma',
 'acritarch',
 'acritarchs',
 'acropora',
 'acrozone',
 'actinides',
 'actinolite',
 'activit',
 'actuelle',
 'actuelles',
 'actuels',
 'ada',
 'adak',
 'adambulacrum',
 'adamellite',
 'adams',
 'adavale',
 'adaville']

In [48]:
geowords_split[32]

'acoma'

In [74]:
"sandstone" in geowords_split

False

In [49]:
type(geowords_split)

list

In [50]:
def onlyEnglishwordsPlease(embedding,english_words):
    new_embedding = {}
    words_in_embedding = list(embedding.keys())
    for word in english_words:
        if word in words_in_embedding:
            new_embedding[word] = embedding[word]
    return new_embedding

In [51]:
geovec_english_embedding = onlyEnglishwordsPlease(new_vector_embedding,geowords_split)

In [52]:
len(list(geovec_english_embedding.keys()))

4587

In [53]:
list(geovec_english_embedding.keys())[0:10]

['ngaterian',
 'congo',
 'nevada',
 'viburnum',
 'advection',
 'turonian',
 'floristics',
 'skilak',
 'phenetic',
 'springhill']

In [55]:
list(geovec_english_embedding.keys())[250:370]

['cambro',
 'speculatively',
 'clade',
 'hampshire',
 'metabasalt',
 'gebco',
 'tyrrhenian',
 'zimbabwe',
 'almandine',
 'malawi',
 'chihuahua',
 'missourian',
 'recharge',
 'baluchistan',
 'barril',
 'kii',
 'escarpment',
 'vocontian',
 'ordovician',
 'dacite',
 'continentaux',
 'vergons',
 'panhandle',
 'sargasso',
 'adelaide',
 'synform',
 'ethiopia',
 'vapeur',
 'kenai',
 'pliny',
 'lycopsida',
 'orthosilicates',
 'swedish',
 'hirnantian',
 'abrupts',
 'xonotlite',
 'gangolihat',
 'artinskian',
 'oldhamia',
 'diagenetic',
 'cladistics',
 'pangean',
 'suboceanic',
 'ferricretes',
 'ghyben',
 'quelques',
 'fourier',
 'hyco',
 'piedra',
 'tectono',
 'succesional',
 'salem',
 'myriophyllum',
 'gallatin',
 'archaean',
 'palaeotemperatures',
 'groupe',
 'depositional',
 'hypersaline',
 'bolivina',
 'scythian',
 'leone',
 'mitteilungen',
 'saskatchewan',
 'anisian',
 'lithofacies',
 'litho',
 'paleogeography',
 'echinodermata',
 'guaymas',
 'cathodoluminescence',
 'siskiyou',
 'meltwater'

In [65]:
geovec_english_embedding['quartz']

KeyError: 'quartz'

In [97]:
def checkIfWordsExistInEmbedding(embedding,words):
    embedding_words = list(embedding.keys())
    for word in words:
        if(word in embedding_words):
            print(word," exists in embedding")
        else:
            print(word," DOES NOT EXIST IN EMBEDDING")

In [102]:
words_I_think_should_be_there = ["sandstone","limestone","bed","fault","fracture","volcano","tuff","strike","dip","line"]

In [103]:
checkIfWordsExistInEmbedding(geovec_english_embedding,words_I_think_should_be_there)

sandstone  DOES NOT EXIST IN EMBEDDING
limestone  DOES NOT EXIST IN EMBEDDING
bed  DOES NOT EXIST IN EMBEDDING
fault  DOES NOT EXIST IN EMBEDDING
fracture  DOES NOT EXIST IN EMBEDDING
volcano  DOES NOT EXIST IN EMBEDDING
tuff  DOES NOT EXIST IN EMBEDDING
strike  DOES NOT EXIST IN EMBEDDING
dip  DOES NOT EXIST IN EMBEDDING
line  DOES NOT EXIST IN EMBEDDING


### Okay, that unfortunately leaves out some obvious geology terms so ~5000 is too small.

### Let's go big and just use a text file of english words to limit the geovec embedding

https://raw.githubusercontent.com/dwyl/english-words/master/words_dictionary.json

In [104]:
import requests

In [105]:
url_to_eng_words_as_json = "https://raw.githubusercontent.com/dwyl/english-words/master/words_dictionary.json"

In [106]:
eng_words_as_json_r  = requests.get(url_to_eng_words_as_json )


In [107]:
eng_words_as_json = eng_words_as_json_r.json()

In [108]:
list(eng_words_as_json.keys())[0:3]

['juergen', 'isocyanic', 'wharve']

In [109]:
len(eng_words_as_json)

370101

In [110]:
len(new_vector_embedding)

411845

In [111]:
def limitGeovecEmbeddingToEnglishWords(new_vector_embedding,eng_words_as_json):
    english_words = list(eng_words_as_json.keys())
    new_english_only_embedding = {}
    for word in english_words:
        if word in new_vector_embedding:
            new_english_only_embedding[word] = new_vector_embedding[word]
    return new_english_only_embedding

In [112]:
new_vector_embedding_english_only2 = limitGeovecEmbeddingToEnglishWords(new_vector_embedding,eng_words_as_json)

In [113]:
len(new_vector_embedding_english_only2)

88626

In [114]:
checkIfWordsExistInEmbedding(new_vector_embedding_english_only2,words_I_think_should_be_there)

sandstone  exists in embedding
limestone  exists in embedding
bed  exists in embedding
fault  exists in embedding
fracture  exists in embedding
volcano  exists in embedding
tuff  exists in embedding
strike  exists in embedding
dip  exists in embedding
line  exists in embedding


In [116]:
list(new_vector_embedding_english_only2.keys())[500:520]

['phiomia',
 'indent',
 'pasiphae',
 'turnerian',
 'pipa',
 'mtg',
 'predicament',
 'corduroy',
 'birdie',
 'hentriacontane',
 'rhinosporidium',
 'unseating',
 'crit',
 'aleuritic',
 'ordovian',
 'delude',
 'asymbiotic',
 'lakeward',
 'ernie',
 'carabinieri']

### Much better result!!!

### Now let's remake the vector and metadata for the embedding projector

In [117]:
def makeVectorTSV(embeddingVectorsOnly):
    out_v = io.open('log/vecs.tsv', 'w', encoding='utf-8')
    for n,weights in enumerate(embeddingVectorsOnly):
        vec = embeddingVectorsOnly[n]
        out_v.write('\t'.join([str(x) for x in vec]) + "\n")

In [128]:
def makeMetaTSV(words):
    with open('log/metadata1.tsv', 'w') as f:
#     for word in word2vec.words:
        print(len(words),"length of words")
        for word in words:
            f.write(word + '\n')

In [132]:
def writeEmbeddingWithWordsAndVectorsToFilesForEmbeddingProjector(embedding):
    vectors_only = []
    keys_in_array = list(embedding.keys())
    for key in keys_in_array:
        vectors_only.append(embedding[key])
    makeVectorTSV(vectors_only)
    makeMetaTSV(keys_in_array)
    print("done")

In [133]:
writeEmbeddingWithWordsAndVectorsToFilesForEmbeddingProjector(new_vector_embedding_english_only2)

88626 length of words
done


### TRY

In [135]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

%load_ext tensorboard


In [137]:
import os
import tensorflow as tf

from tensorboard.plugins import projector


TypeError: __init__() got an unexpected keyword argument 'serialized_options'